In [77]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import math
import os

In [78]:
# Directory of melanoma dataset
dir = "D:\\Mega\\Doutorado-UFG\\Disciplinas\\Processamento Digital de Imagem\\Artigo\\complete_mednode_dataset\\dataset\\"

In [79]:
# Function to convert float number in float32 number
def np_hist_to_cv(np_histogram_output):
    counts, bin_edges = np_histogram_output
    return counts.ravel().astype('float32')

In [101]:
def extractFeatures(image_gray, radius):
    pi = 3.14

    G_x = cv2.reduce(image_gray/255, 0, cv2.REDUCE_SUM)
    G_y = cv2.reduce(image_gray/255, 1, cv2.REDUCE_SUM)
    h1 = np.histogram(G_x)
    h2 = np.histogram(G_y.T)

    # A feature - Assymetry (less is more assymetric)
    a = cv2.compareHist(np_hist_to_cv(h1), np_hist_to_cv(h2), cv2.HISTCMP_CORREL)
    # print(assymetry)

    # B feature - Border
    perimeter = 2 * pi * (radius/100)
    roundness = 4 * pi * (radius/100)/(perimeter**2)
    # print(roundness)

    # C feature - Color
    r = image[:, :, 0].mean()
    g = image[:, :, 1].mean()
    b = image[:, :, 2].mean()

    # D feature - Diameter
    d = math.sqrt((4 * (pi * ((radius/100)**2)))/pi)
    return a, d, r, g, b, roundness

In [132]:
dataset = pd.DataFrame(columns= ["assymetry", "border", "diameter", "color_r", "color_g", "color_b", "label"])

for path,d,f in os.walk(dir):
    if len(f) > 0:  # verifica se f é um arquivo válido
        for i in f:
            if i.endswith('.jpg') or i.endswith('.png'):
                img = path + '\\' +i
                image = cv2.imread(img)
                # Convert RGB image to grayscale
                image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                # Generate histogram of projection in X and Y axes to calculate assymetry
                contours, hierarchy = cv2.findContours(image_gray, 2, 1)
                # print(len(contours))
                cnt = contours
                for j in range(1):
                    (x, y), radius = cv2.minEnclosingCircle(cnt[j])
                    center = (int(x), int(y))
                    radius = int(radius)
                    # plt.text(x-21, y+15, '+', fontsize=25, color='red')
                    # plt.text(10, -10, 'Centro: ' +str(center), fontsize=11, color='red')
                    # plt.text(840, -10, 'Diâmetro: ' + str((radius*2)/100)+'mm', fontsize=11, color='red')
                    # plt.Circle((10, -10), 7.2, color='blue')
                    # plt.imshow(image_gray, cmap='gray')
                    # plt.show()
                    a, d, r, g, b, roundness = extractFeatures(image_gray, radius)
                    if img.__contains__('melanoma'):
                        dataset.loc[len(dataset)] = [a, roundness, d, r, g, b, 1]
                    else:
                        dataset.loc[len(dataset)] = [a, roundness, d, r, g, b, 0]

dataset.to_csv('dataset_melanoma.csv')

In [135]:
# Classification of dataset
from sklearn.feature_selection import RFECV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

labels = dataset.iloc[:, 6]
datasets = dataset.drop(columns=['label'], axis=1)

selector = RFECV(LinearSVC(), cv=10, scoring='f1_weighted')
sm = SMOTE(random_state=42) # Oversampling
#sm = NearMiss(random_state=42) # Undersampling

datasets, labels = sm.fit_resample(datasets,labels)

train, test, train_tlabels, test_labels = train_test_split(datasets,
                                                           labels,
                                                           test_size=0.33)


model = LinearSVC()
model.fit(train, train_tlabels)
y_pred = model.predict(test)

# Evaluation
acc = accuracy_score(test_labels, y_pred)
print("Accuracy: {:.2f}".format(acc*100))

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Accuracy: 77.61


Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_38_64.pyx", line 1035, in _pydevd_bundle.pydevd_cython_win32_38_64.PyDBFrame.trace_dispatch
  File "D:\Arquivo de Programas\DataSpell 2021.3.1\plugins\python-ce\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "D:\Arquivo de Programas\DataSpell 2021.3.1\plugins\python-ce\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "D:\Arquivo de Programas\DataSpell 2021.3.1\plugins\python-ce\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined
